In [ ]:
# Configuration - Docker container paths
BASE_DIR = "/nnv/code/nnv/examples/Submission/FORMALISE2025"
DATA_DIR = f"{BASE_DIR}/data"
MODELS_DIR = f"{BASE_DIR}/models"

# Downloading UCF11 Dataset

In [2]:
!apt-get install unrar -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [3]:
# download ucf11 dataset
!wget --no-check-certificate http://crcv.ucf.edu/data/UCF11_updated_mpg.rar
!unrar x UCF11_updated_mpg.rar

--2025-09-09 02:17:09--  http://crcv.ucf.edu/data/UCF11_updated_mpg.rar
Resolving crcv.ucf.edu (crcv.ucf.edu)... 132.170.214.127
Connecting to crcv.ucf.edu (crcv.ucf.edu)|132.170.214.127|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.crcv.ucf.edu/data/UCF11_updated_mpg.rar [following]
--2025-09-09 02:17:09--  https://www.crcv.ucf.edu/data/UCF11_updated_mpg.rar
Resolving www.crcv.ucf.edu (www.crcv.ucf.edu)... 132.170.214.127
Connecting to www.crcv.ucf.edu (www.crcv.ucf.edu)|132.170.214.127|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 1045106394 (997M) [application/x-rar-compressed]
Saving to: ‘UCF11_updated_mpg.rar’

UCF11_updated_mpg.r 100%[===================>] 996.69M  76.7MB/s    in 11s     

2025-09-09 02:17:20 (91.3 MB/s) - ‘UCF11_updated_mpg.rar’ saved [1045106394/1045106394]


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Rosh

# Process UCF11 into .npy file

In [4]:
import numpy as np
import cv2
import os
from tqdm import tqdm

In [5]:
data_path = "UCF11_updated_mpg"
classes = sorted(os.listdir(data_path))
class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}

In [12]:
FRAME_SIZE = (112, 112)  # Resize all frames to 112x112
FRAMES_PER_VIDEO = 32    # Number of frames per video (you can adjust this)

In [13]:
def load_video_frames(video_path, max_frames=FRAMES_PER_VIDEO):
    cap = cv2.VideoCapture(video_path)
    frames = []
    # while cap.isOpened() and len(frames) < max_frames:
    #     ret, frame = cap.read()
    #     if not ret:
    #         break
    #     frame = cv2.resize(frame, FRAME_SIZE)
    #     frames.append(frame)
    # cap.release()

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
          break
        frame = cv2.resize(frame, FRAME_SIZE)
        frames.append(frame)

        # h, w = frame.shape[:2]

        # start_x = max(0, (w - 112) // 2)
        # start_y = max(0, (h - 112) // 2)

        # # Calculate ending coordinates for the crop
        # end_x = min(w, start_x + 112)
        # end_y = min(h, start_y + 112)

        # # Perform the crop using NumPy slicing
        # cropped_frame = frame[start_y:end_y, start_x:end_x]

        # frames.append(cropped_frame)

    cap.release()

    # interpolate to get subset of frames
    idx = np.linspace(0, len(frames) - 1, max_frames, dtype=int)
    final_frames = [frames[i] for i in idx]

    # If video is too short, pad with last frame
    while len(final_frames) < max_frames:
        final_frames.append(frames[-1])

    return np.array(final_frames, dtype=np.uint8)

In [14]:
data = []
labels = []

for cls in tqdm(classes, desc="Processing Classes"):
    class_dir = os.path.join(data_path, cls)
    for group in os.listdir(class_dir):
        if group == "Annotation":
            continue
        group_dir = os.path.join(class_dir, group)
        for video_file in os.listdir(group_dir):
            video_path = os.path.join(group_dir, video_file)
            frames = load_video_frames(video_path)
            data.append(frames)
            labels.append(class_to_idx[cls])

Processing Classes: 100%|██████████| 11/11 [01:51<00:00, 10.12s/it]


In [17]:
# check an instance of the dataset to verify it looks good
import imageio.v3 as iio
from IPython.display import Video
iio.imwrite("/content/out.mp4", data[0], fps=16, codec="libx264", macro_block_size=None)
Video("/content/out.mp4", embed=True)

In [18]:
data_array = np.array(data, dtype=np.uint8)      # Shape: [num_videos, frames, height, width, channels]
labels_array = np.array(labels, dtype=np.int32)  # Shape: [num_videos]

print("Data shape:", data_array.shape)
print("Labels shape:", labels_array.shape)

Data shape: (1600, 32, 112, 112, 3)
Labels shape: (1600,)


In [ ]:
import os
save_dir = os.path.join(DATA_DIR, 'UCF11', 'original')
os.makedirs(save_dir, exist_ok=True)
np.save(os.path.join(save_dir, f"ucf11_data_{FRAMES_PER_VIDEO}f.npy"), data_array)
np.save(os.path.join(save_dir, f"ucf11_labels_{FRAMES_PER_VIDEO}f.npy"), labels_array)

print("Saved ucf11_data.npy and ucf11_labels.npy")